In [2]:
# !conda install -c conda-forge jupyterlab-drawio

# <b>Diagramas e mapas</b>

## Dimensões sobre pesquisa em Inteligência Artificial (IA)

``` mermaid 
graph LR
    subgraph Racionalmente
        RL(Sistemas para Raciocinar logicamente e gerar informações)
        AO(Sistemas para Agir e atingir objetivos):::strong
    end

    subgraph Humanamente
        PI(Sistemas para Processar informação como humanos)
        CH(Sistemas para Emular ações humanas de comportamento)
    end

    P[Pensar]:::pensar -.-> PI & RL
    A[Agir]:::agir -.-> CH & AO

    style P fill:#f2e6ff,stroke:#9966ff,stroke-width:2px, color:#9966ff
    style A fill:#e6f2ff,stroke:#6699ff,stroke-width:2px, color:#6699ff

    classDef humanamente fill:#f9f,stroke:#333,stroke-width:2px;
    classDef strong fill:#ccf,stroke:#333,stroke-width:4px;

    linkStyle 0,1 stroke:#333,stroke-width:1px;

    style PI fill:#f2e6ff,stroke:#9966ff,stroke-width:2px
    style CH fill:#e6f2ff,stroke:#6699ff,stroke-width:2px
    style RL fill:#f2e6ff,stroke:#9966ff,stroke-width:2px
    style AO fill:#e6f2ff,stroke:#6699ff,stroke-width:2px
```

## Mapas mentais sobre pesquisa em IA

``` mermaid
mindmap
  root((Temas em Inteligência Artificial))
    (Representação de Conhecimento)
        Ontologias
        Redes Semânticas
        Lógica de Primeira Ordem
        Grafos de Conhecimento
    (Resolução de Problemas)
        Busca em Grafos
        Planejamento
        Jogos
    (Outras Áreas)
        Visão Computacional
        Geração de Texto
        Robótica
        ...
    (Temas Transversais)
        Ética e IA
        Segurança em IA
        IA e Sociedade
        ...
```

``` mermaid
mindmap
  root((Bases para Inteligência Artificial - IA))
    (Fundamentos)
        (Representação de Conhecimento e Raciocínio)
        (Resolução de problemas)
        (Aprendizado de Máquina)
        (Processamento de Linguagem Natural - PLN)
        (Modelos em Grafos)
    (Áreas de Aplicação)
        Sistemas Especialistas
        Visão Computacional
        Bioinformática
        Automação
        Finanças
        Jogos
    (Abordagens Metodológicas)
        Deep Learning
        Aprendizado Estruturado
        Computação Evolutiva
        Lógica Fuzzy
```

``` mermaid
mindmap
  root((Inteligência Artificial - IA))
    (Fundamentos)
        (Representação de Conhecimento e Raciocínio)
            Lógica de primeira ordem
            Redes Semânticas
            Ontologias
            Raciocínio Probabilístico
        (Resolução de problemas)
            Busca e Planejamento
            Algoritmos de Busca
            Planejamento Clássico
            Planejamento em Ambientes Incertos
        (Aprendizado de Máquina)
            Aprendizado Supervisionado
            Aprendizado Não Supervisionado
            Aprendizado por Reforço
            Aprendizado Profundo - Deep Learning
            Aprendizado Estruturado
        (Processamento de Linguagem Natural - PLN)
            Compreensão de Linguagem Natural
            Geração de Linguagem Natural
            Tradução Automática
            Análise de Sentimentos
        (Modelos em Grafos)
            Redes Bayesianas
            Modelos Markovianos Ocultos - HMMs
            Conditional Random Fields - CRFs
            Grafos de Conhecimento
            Redes Neurais para Grafos - GNNs
    (Áreas de Aplicação)
        Visão Computacional
            Reconhecimento de Imagens
            Detecção de Objetos
            Segmentação de Imagens
        Robótica
            Navegação
            Planejamento de Movimento
            Controle de Robôs
            Interação Humano-Robô
        Sistemas Especialistas
        Jogos
        Bioinformática
        Finanças
    (Abordagens Metodológicas)
        Deep Learning
            Redes Neurais Artificiais
        Aprendizado Estruturado
        Computação Evolutiva
        Lógica Fuzzy
    (Temas Transversais)
        Ética e IA
        Segurança em IA
        IA e Sociedade
```

## Mapa mental em análises de grafos

``` mermaid
mindmap
  root((Modelagem do Mundo Real em Grafos para Aplicações em Saúde))
    (Grafos Simples)
      Aplicações
        Interações Medicamento-Medicamento - Efeitos Adversos
        Co-ocorrência de Doenças - Comorbidades
      Limitações
        Não modela a direção ou força das relações
    (Grafos Direcionados)
      Aplicações
        Propagação de Doenças Infecciosas - Transmissão
        Vias Metabólicas - Reações Bioquímicas
      Limitações
        Não modela múltiplas relações entre os mesmos elementos
    (Grafos Ponderados)
      Aplicações
        Redes de Interação Proteína-Proteína - Afinidade de Ligação
        Redes de Coexpressão Gênica - Correlação
      Limitações
        Não modela diferentes tipos de relações simultaneamente
    (Grafos Multicamadas)
      Aplicações
        Redes de Interação Fármaco-Alvo-Doença Múltiplos Mecanismos
        Redes de Microbioma Humano 
        Nichos Tecnológicos
      Limitações
        Aumenta a complexidade da análise
    (Hipergrafos)
      Aplicações
        Análise de Dados de Saúde Multimodais - Prontuários, Imagens, Genética
        Modelagem de Fatores de Risco Complexos para Doenças Multifatoriais
      Limitações
        Dificuldade de visualização e análise
```

## Diagramas de blocos da metodologia

``` mermaid
graph BT
        R(Feedback)
    subgraph Extrair_e_preparar_dados_de_entrada
        A[(Currículos Pesquisadores da ICT)]
        B[(Intenções dos Pesquisadores da ICT)]
        C[Processos em PDI]
        D[Produtos Estratégicos CEIS]
        E[Limpar e Normalizar Termos]
    end

    subgraph Gerar Embeddings em GPU
        H[Embedding Multilingue Sentence Transformers]
        I[Codificar Curriculos e Interesses dos Pesquisadores]
        J[Codificar Produtos estratégicos para o CEIS]
    end

    subgraph Modelar_Grafo_de_Conhecimento
        K[Gerar Entidades e Relacionamentos]
        L[Grafo Multiplex dos Macroprocessos PDI no CEIS]
    end

    subgraph Analisar_Modelo_Grafo_com_ML/DL
        M[Detectar Comunidades]
        N[Analisar Similaridade com Redes Neurais em Grafos]
        O[Identificar Lacunas]
    end

    subgraph Visualizar e Recomendar
        P[Recomendar Alinhamento Competências/Produtos]
        Q[Visualizar Resultados]
    end

    A --> E
    B --> E
    C --> E
    D --> E
    E --> H
    H --> I
    H --> J
    I --> K
    J --> K
    N <--> M
    L --> M
    L --> N    
    M --> O
    N --> O
    K --> L
    O --> P
    P --> Q
    P -.- R
    Q -.- R
    R -.- L
    R -.- J
    R -.- I
```

``` mermaid
graph BT
    R(Feedback)
    subgraph A._Extract_and_prepare_input_data
        A[(ICT Researchers' Resumes)]
        B[(ICT Researchers' Intentions)]
        C[Processes in R&D]
        D[CEIS Strategic Products]
        E[Clean and Normalize Terms]
    end

    subgraph B._Generate_Embeddings_in_GPU
        H[Multilingual Sentence Transformers Embedding]
        I[Encode Researchers' Resumes and Interests]
        J[Encode Strategic Products for CEIS]
    end

    subgraph C._Model_Knowledge_Graph
        K[Generate Entities and Relationships]
        L[Multilayer Graph of RDI Macroprocesses at CEIS]
    end

    subgraph D._Analyze_Graph_Model_with_ML/DL
        M[Detect Communities]
        N[Analyze Similarity with Graph Neural Networks]
        O[Identify Gaps]
    end

    subgraph E._Visualize_and_Recommend
        P[Recommend Skills/Products Alignment]
        Q[Visualize Results]
    end

    A --> E
    B --> E
    C --> E
    D --> E
    E --> H
    H --> I
    H --> J
    I --> K
    J --> K
    N <--> M
    L --> M
    L --> N    
    M --> O
    N --> O
    K --> L
    O --> P
    P --> Q
    P -.- R
    Q -.- R
    R -.- L
    R -.- J
    R -.- I
```

## Visualização dos fluxos de dados em processamento

```mermaid
graph LR
    CPU <--> Barramento;
    GPU <--> Barramento;
    Barramento <--> RAM;
    SSD <--> Barramento;

    subgraph " "
        CPU[CPU]:::cpu
        GPU[GPU]:::gpu
        RAM[Memória RAM]:::ram
        SSD[Armazenamento]:::ssd
        Barramento[Barramento]:::bus
    end
    
    classDef cpu fill:#f9f,stroke:#333,stroke-width:2px;
    classDef gpu fill:#ccf,stroke:#333,stroke-width:2px;
    classDef ram fill:#ff9,stroke:#333,stroke-width:2px;
    classDef ssd fill:#f0f,stroke:#333,stroke-width:2px;
    classDef bus fill:#9fc,stroke:#333,stroke-width:2px;
```

```mermaid
graph LR
    CPU_Processador -. Leitura .- Barramento;
    CPU_Processador -. Escrita .- Barramento;
    GPU_Aceleradora -. Leitura .- Barramento;
    GPU_Aceleradora -. Escrita .- Barramento;
    SSD -. Leitura .- Barramento;
    SSD -. Escrita .- Barramento;
    RAM -. Leitura .- Barramento;
    RAM -. Escrita .- Barramento; 
    Barramento -. Leitura .- RAM;
    Barramento -. Escrita .- RAM;        

    subgraph " "
        CPU_Processador[CPU_Processador]:::cpu
        GPU_Aceleradora[GPU_Aceleradora]:::gpu
        RAM[Memória RAM]:::ram
        SSD[Armazenamento]:::ssd
        Barramento[====__Barramento_PCIexpress__====]:::bus
    end
    
    classDef cpu fill:#f9f,stroke:#333,stroke-width:2px;
    classDef gpu fill:#ccf,stroke:#333,stroke-width:2px;
    classDef ram fill:#ff9,stroke:#333,stroke-width:2px;
    classDef ssd fill:#f0f,stroke:#333,stroke-width:2px;
    classDef bus fill:#9fc,stroke:#333,stroke-width:2px;
```

```mermaid
graph LR
    CPU --> Barramento_PCIe;
    GPU --> Barramento_PCIe;
    Barramento_PCIe <--> RAM;
    SSD1 --> Barramento_PCIe;
    SSD2 --> Barramento_SATA;
    Barramento_SATA --> Ponte_Southbridge;
    Ponte_Southbridge --> Barramento_PCIe;
    USB --> Ponte_Southbridge;

    subgraph " "
        CPU[CPU]:::cpu
        GPU[GPU]:::gpu
        RAM[Memória RAM]:::ram
        SSD1[SSD NVMe]:::ssd
        SSD2[SSD SATA]:::ssd
        Barramento_PCIe[====__Barramento PCIe__====]:::bus
        Barramento_SATA[---- Barramento SATA ----]:::bus
        Ponte_Southbridge[Ponte Southbridge]:::chipset
        USB[USB]:::usb
    end
    
    classDef cpu fill:#f9f,stroke:#333,stroke-width:2px;
    classDef gpu fill:#ccf,stroke:#333,stroke-width:2px;
    classDef ram fill:#ff9,stroke:#333,stroke-width:2px;
    classDef ssd fill:#f0f,stroke:#333,stroke-width:2px;
    classDef bus fill:#9fc,stroke:#333,stroke-width:2px;
    classDef chipset fill:#ccc,stroke:#333,stroke-width:2px;
    classDef usb fill:#9ff,stroke:#333,stroke-width:2px;
```

```mermaid
graph LR
    CPU -->|"Leitura/Escrita"| Barramento_PCIe;
    GPU -->|"Leitura/Escrita"| Barramento_PCIe;
    Barramento_PCIe <-->|"Leitura/Escrita"| RAM;
    SSD1 -->|"Leitura/Escrita"| Barramento_PCIe;
    SSD2 -->|"Leitura/Escrita"| Barramento_SATA;
    Barramento_SATA --> Ponte_Southbridge;
    Ponte_Southbridge --> Barramento_PCIe;
    USB --> Ponte_Southbridge;

    subgraph " "
        CPU[CPU]:::cpu
        GPU[GPU]:::gpu
        RAM[Memória RAM]:::ram
        SSD1[SSD NVMe]:::ssd
        SSD2[SSD SATA]:::ssd
        Barramento_PCIe[====__Barramento PCIe__====]:::bus
        Barramento_SATA[---- Barramento SATA ----]:::bus
        Ponte_Southbridge[Ponte Southbridge]:::chipset
        USB[USB]:::usb
    end
    
    classDef cpu fill:#f9f,stroke:#333,stroke-width:2px;
    classDef gpu fill:#ccf,stroke:#333,stroke-width:2px;
    classDef ram fill:#ff9,stroke:#333,stroke-width:2px;
    classDef ssd fill:#f0f,stroke:#333,stroke-width:2px;
    classDef bus fill:#9fc,stroke:#333,stroke-width:2px;
    classDef chipset fill:#ccc,stroke:#333,stroke-width:2px;
    classDef usb fill:#9ff,stroke:#333,stroke-width:2px;
```

```mermaid
graph LR
    CPU -->|"até 16 GB/s"| Barramento_PCIe;
    GPU -->|"até 16 GB/s"| Barramento_PCIe;
    Barramento_PCIe <-->|"até 64 GB/s"| RAM;
    SSD1 -->|"até 4 GB/s"| Barramento_PCIe;
    SSD2 -->|"até 0,6 GB/s"| Barramento_SATA;
    Barramento_SATA -->|"até 0,55 GB/s"| Ponte_Southbridge;
    Ponte_Southbridge -->|"até 5 GB/s"| Barramento_PCIe;
    USB -->|"até 10 Gbps"| Ponte_Southbridge;

    subgraph " "
        CPU[CPU]:::cpu
        GPU[GPU]:::gpu
        RAM[Memória RAM]:::ram
        SSD1[SSD NVMe]:::ssd
        SSD2[SSD SATA]:::ssd
        Barramento_PCIe[====__Barramento PCIe__====]:::bus
        Barramento_SATA[---- Barramento SATA ----]:::bus
        Ponte_Southbridge[Ponte Southbridge]:::chipset
        USB[USB]:::usb
    end
    
    classDef cpu fill:#f9f,stroke:#333,stroke-width:2px;
    classDef gpu fill:#ccf,stroke:#333,stroke-width:2px;
    classDef ram fill:#ff9,stroke:#333,stroke-width:2px;
    classDef ssd fill:#f0f,stroke:#333,stroke-width:2px;
    classDef bus fill:#9fc,stroke:#333,stroke-width:2px;
    classDef chipset fill:#ccc,stroke:#333,stroke-width:2px;
    classDef usb fill:#9ff,stroke:#333,stroke-width:2px;
```

Na maioria dos casos, a comunicação entre CPU e GPU não é totalmente direta, mas sim mediada pela memória RAM e pelo barramento PCIe. Esse é o cenário mais comum em computadores pessoais e servidores convencionais.

Como funciona a comunicação "indireta" via RAM:

- A CPU processa dados e os envia para a memória RAM.
- A GPU acessa esses dados na RAM através do barramento PCIe.
- A GPU processa os dados (ex: renderiza gráficos) e grava os resultados na RAM.
- A CPU acessa os resultados na RAM, completando o ciclo.

Vantagens da comunicação via RAM:

- Flexibilidade: Permite que múltiplas CPUs e GPUs acessem e compartilhem dados de forma organizada.
- Escalabilidade: Facilita a expansão do sistema com mais memória ou GPUs.
- Custo-benefício: É uma solução mais econômica em comparação com tecnologias de comunicação direta.

Quando há comunicação direta?

Em alguns casos específicos, existe sim comunicação direta entre CPU e GPU, sem a necessidade de passar pela RAM. Isso ocorre principalmente em sistemas de alto desempenho e para tarefas que exigem altíssima velocidade de transferência de dados, como:

- Inteligência Artificial: Treinamento de modelos de deep learning e inferência em larga escala.
- Computação Científica: Simulações complexas, análise de dados massivos, etc.
- Processamento de imagens e vídeos: Edição e renderização de vídeos em alta resolução, etc.

Tecnologias que permitem comunicação direta:

- NVLink (NVIDIA): Conecta CPUs e GPUs NVIDIA com alta velocidade e baixa latência.
- AMD Infinity Fabric: Interconecta CPUs, GPUs e outros componentes AMD.

Ou seja, embora a comunicação direta entre CPU e GPU seja possível e vantajosa em cenários específicos, a comunicação via RAM e barramento PCIe ainda é a mais comum e eficiente na maioria das situações de computadores pessoais. A escolha da melhor abordagem depende dos requisitos de desempenho, custo e complexidade do sistema.

```mermaid
graph LR
    CPU -->|"Leitura/Escrita"| Barramento_PCIe;
    GPU -->|"Leitura/Escrita"| Barramento_PCIe;
    Barramento_PCIe <-->|"Leitura/Escrita"| RAM;
    SSD1 -->|"Leitura/Escrita"| Barramento_PCIe;
    SSD2 -->|"Leitura/Escrita"| Barramento_SATA;
    Barramento_SATA --> Ponte_Southbridge;
    Ponte_Southbridge --> Barramento_PCIe;
    USB --> Ponte_Southbridge;
    CPU <-->|"NVLink/Infinity Fabric"| GPU;

    subgraph " "
        CPU[CPU]:::cpu
        GPU[GPU]:::gpu
        RAM[Memória RAM]:::ram
        SSD1[SSD NVMe]:::ssd
        SSD2[SSD SATA]:::ssd
        Barramento_PCIe[====__Barramento PCIe__====]:::bus
        Barramento_SATA[---- Barramento SATA ----]:::bus
        Ponte_Southbridge[Ponte Southbridge]:::chipset
        USB[USB]:::usb
    end
    
    classDef cpu fill:#f9f,stroke:#333,stroke-width:2px;
    classDef gpu fill:#ccf,stroke:#333,stroke-width:2px;
    classDef ram fill:#ff9,stroke:#333,stroke-width:2px;
    classDef ssd fill:#f0f,stroke:#333,stroke-width:2px;
    classDef bus fill:#9fc,stroke:#333,stroke-width:2px;
    classDef chipset fill:#ccc,stroke:#333,stroke-width:2px;
    classDef usb fill:#9ff,stroke:#333,stroke-width:2px;
```

Em um sistema pessoal real os dados são:

- Velocidade da RAM: Velocidade máxima da RAM para 128 GB/s, pois a X570 suporta essa velocidade com overclocking.
- PCIe 4.0: Versão do PCIe (4.0) no rótulo do barramento.
- SATA III: Versão do SATA (III) no rótulo do barramento.
- Ponte Southbridge: Em vez do nome genérico "Ponte Southbridge" é designado pelo chipset específico da placa mãe (X570).

Observações:

- Limitações: Mesmo com o PCIe 4.0, a velocidade de transferência de dados pode ser limitada por outros fatores, como a velocidade da RAM, do SSD e do processador.
- CPU: Velocidade máxima do PCIe da CPU em 32 GB/s, dado que o Ryzen 7 5800X possui 20 lanes PCIe 4.0, que são divididas entre o slot da placa de vídeo (x16) e outros dispositivos.
- Overclocking: A velocidade máxima da RAM de 128 GB/s pode ser atingida com overclocking. A velocidade real depende tando da configuração de memória na BIOS como das capacidades da placa mãe.

Este diagrama atualizado reflete as velocidades de transferência da placa mãe Gigabyte X570 AORUS ULTRA, que equipa o ambeiente de testes.

```mermaid
graph LR
    CPU -->|"PCIe 4.0 x16 até 32 GB/s"| Barramento_PCIe;
    GPU -->|"PCIe 4.0 x16 até 32 GB/s"| Barramento_PCIe;
    Barramento_PCIe <-->|"DDR4 até 128 GB/s (O.C.)"| RAM;
    SSD1 -->|"PCIe 4.0 x4 até 8 GB/s"| Barramento_PCIe;
    SSD2 -->|"SATA III até 600 MB/s"| Barramento_SATA;
    Barramento_SATA -->|"até 600 MB/s"| Ponte_Southbridge;
    Ponte_Southbridge -->|"PCIe 4.0 x4 até 8 GB/s"| Barramento_PCIe;
    USB -->|"USB 3.2 Gen2x2 até 20 Gbps"| Ponte_Southbridge;

    subgraph " "
        CPU[Ryzen 7 5800X]:::cpu
        GPU[RTX 4080]:::gpu
        RAM[64GB DDR4]:::ram
        SSD1[SSD NVMe]:::ssd
        SSD2[SSD SATA]:::ssd
        Barramento_PCIe[====__Barramento_PCIe_4.0__====]:::bus
        Barramento_SATA[---- Barramento SATA III ----]:::bus
        Ponte_Southbridge[Chipset X570]:::chipset
        USB[USB]:::usb
    end
    
    classDef cpu fill:#f9f,stroke:#333,stroke-width:2px;
    classDef gpu fill:#ccf,stroke:#333,stroke-width:2px;
    classDef ram fill:#ff9,stroke:#333,stroke-width:2px;
    classDef ssd fill:#f0f,stroke:#333,stroke-width:2px;
    classDef bus fill:#9fc,stroke:#333,stroke-width:2px;
    classDef chipset fill:#ccc,stroke:#333,stroke-width:2px;
    classDef usb fill:#9ff,stroke:#333,stroke-width:2px;
```

# <b>Etapas da Metodologia</b>

## Etapas para gerar o grafo de conhecimento:

Carregar os dados dos arquivos JSON:

- Carregar os dados dos currículos dos pesquisadores em um DataFrame cuDF.
- Carregar os dados da Matriz CEIS em um DataFrame cuDF.
- Carregar as relações para biológicos e pequenas moléculas em estruturas de dados Python (listas de dicionários).

Criar o grafo de conhecimento usando aceleração de hardware:
- Criar um grafo direcionado cuGraph.
- Ingerir nós de Pesquisador, Produto CEIS e Etapas de desenvolvimento (biológicos e pequenas moléculas).
- Ingerir atributos para os nós e arestas, como tipo de nó ("pesquisador", "produto", "relacionamento"), nome, ID, etc.

Conectar os nós através dos relacionamentos:
- Ingerir arestas para representar os relacionamentos entre as diversas entidades.
- Conectar os nós de etapas entre si, seguindo as relações definidas nos arquivos JSON de bioprodutos e pequenas moléculas.
- Conectar nós de Pesquisadores aos nós Produtos com base na similaridade entre dados do currículo, interesses e áreas de atuação do pesquisador e as Etapas de PDI, áreas de aplicação e necessidades relacionadas ao Produto.
- Conectar os nós de Competências aos nós de Etapas, com base nos relacionamentos demandamos com base no tipo de produto (biológico ou pequena molécula).

Calcular métricas de grafo:
- Calcular métricas de centralidade, como grau de entrada e saída, para identificar os nós mais importantes no grafo.
- Calcular métricas de caminho, como distância e betweenness centrality, para analisar as relações entre os nós.
- Converter o grafo cuGraph para NetworkX:

# Preparar a Ingestão de Dados

Obs.: Foi necessário adaptar o código de extração de nós e arestas da função construct_graph de acordo com a estrutura dos arquivos JSON obtidos da fase de levantamento de dados. Para certificar de que os arquivos JSON estejam sempre no formato correto, contendo informações sobre os nós (features) e as arestas (source e target) necessárias foram utilizaods esquemas de validação para garantir as entradas necessárias ao processamento.

## Otimizar processamento de dados

Otimização do código:

- Vetorização: Foram utilizadas operações vetorizadas no PyTorch para aproveitar ao máximo a capacidade de processamento paralelo da GPU.
- Tamanhos de lote: Foram realizados experimentados com diferentes tamanhos de lote para encontrar o valor ideal que maximiza a utilização da GPU e a velocidade de treinamento.
- Transferência de dados: Foi minimizada a transferência de dados entre a CPU e a GPU, utilizando técnica de pin_memory e DataLoader com múltiplos workers.
- Operações assíncronas: Foram utilizadas operações assíncronas para sobrepor a computação na GPU com a transferência de dados.

A velocidade de treinamento depende de muitos fatores, dentre eles da complexidade do modelo de IA. Modelos mais complexos com mais camadas e parâmetros levam mais tempo para treinar. E por óbvio, quanto maiores os conjuntos de dados, maiores serão os tempo de treinamento, mas nos experimentos foi utilizado o mesmo conjunto de dados para avaliar cada um dos modelos. Além disso, a velocidade de treinamento pode ser afetada por outros fatores, como a otimização do código, a escolha do otimizador e a configuração dos hiperparâmetros para cada modelo.

Detecção de gargalos por comparação com as velocidades teóricas:

- Largura de banda da memória: Comparar a taxa de transferência de dados da RAM (DDR4 até 128 GB/s neste caso) com a taxa de acesso à memória do modelo durante o treinamento. Se a taxa de acesso à memória for significativamente menor que a largura de banda da RAM, pode haver um gargalo de memória.
- Velocidade do barramento PCIe: Comparar a velocidade do barramento PCIe (até 32 GB/s neste caso) com a taxa de transferência de dados entre a CPU e a GPU durante o treinamento. Se a taxa de transferência for muito menor que a velocidade do PCIe, pode haver um gargalo no barramento.
- Velocidade da GPU: Analisar a utilização da GPU (RTX 4080) durante o treinamento. Se a utilização estiver baixa, o modelo pode não estar sendo executado com a máxima eficiência na GPU.

Para superar gargalo de memória,  o código foi otimizado para reduzir o acesso à memória, antes de avaliar a troca por RAM mais rápida; Para manter a utilização da GPU alta, foi aumentado o tamanho do lote e foram utilizadas técnicas de otimização de código para melhorar a performance na GPU. Para isso, foi utilizado PyTorch Profiler para identificar gargalos de desempenho de cada modelo; Foi monitorada a utilização da GPU com o NVIDIA SMI durante o treinamento; e a taxa de acesso à memória do modelo foi coomparada com a largura de banda da RAM.

### Esquemas de validação JSON para dados de entrada

In [10]:
# %pip install validclust

In [ ]:
import os
import json
from git import Repo
from model_evaluation import ModelPerformance, GNNModel, GKANModel, GraphDataset, JSONValidator

repo = Repo(search_parent_directories=True)
root_folder = repo.working_tree_dir

# Esquema JSON para validação (schema.json)
schema_curriculos = {
  "type": "array",
  "items": {
    "type": "object",
    "properties": {
      "Identificação": {
        "type": "object",
        "properties": {
          "Nome": {"type": "string"},
          "ID Lattes": {"type": "string"},
          "Última atualização": {"type": "string"}
        },
        "required": ["Nome", "ID Lattes", "Última atualização"]
      },
      "Idiomas": {
        "type": "array",
        "items": {
          "type": "object",
          "properties": {
            "Idioma": {"type": "string"},
            "Proficiência": {"type": "string"}
          },
          "required": ["Idioma", "Proficiência"]
        }
      },
      "Formação": {
        "type": "object",
        "properties": {
          "Acadêmica": {
            "type": "array",
            "items": {
              "type": "object",
              "properties": {
                "Ano": {"type": "string"},
                "Descrição": {"type": "string"}
              },
              "required": ["Ano", "Descrição"]
            }
          },
          "Pos-Doc": {"type": "array"},
          "Complementar": {
            "type": "array",
            "items": {
              "type": "object",
              "properties": {
                "Ano": {"type": "string"},
                "Descrição": {"type": "string"}
              },
              "required": ["Ano", "Descrição"]
            }
          }
        },
        "required": ["Acadêmica", "Pos-Doc", "Complementar"]
      },
      "Atuação Profissional": {
        "type": "array",
        "items": {
          "type": "object",
          "properties": {
            "Instituição": {"type": "string"},
            "Ano": {"type": "string"},
            "Descrição": {"type": "string"},
            "Outras informações": {"type": "string"}
          },
          "required": ["Instituição", "Ano", "Descrição", "Outras informações"]
        }
      },
      "Linhas de Pesquisa": {"type": "array"},
      "Áreas": {
        "type": "object",
        "patternProperties": {
          "^[0-9]+\\.$": {"type": "string"}
        }
      },
      "Produções": {
        "type": "object",
        "properties": {
          "Artigos completos publicados em periódicos": {
            "type": "array",
            "items": {
              "type": "object",
              "properties": {
                "ano": {"type": "string"},
                "fator_impacto_jcr": {"type": "string"},
                "ISSN": {"type": "string"},
                "titulo": {"type": "string"},
                "revista": {"type": "string"},
                "autores": {"type": "string"},
                "data_issn": {"type": "string"},
                "DOI": {"type": "string"},
                "Qualis": {"type": "string"}
              },
              "required": ["ano", "fator_impacto_jcr", "ISSN", "titulo", "revista", "autores", "data_issn", "DOI", "Qualis"]
            }
          },
          "Resumos publicados em anais de congressos": {
            "type": "object",
            "patternProperties": {
              "^[0-9]+\\.$": {"type": "string"}
            }
          },
          "Apresentações de Trabalho": {
            "type": "object",
            "patternProperties": {
              "^[0-9]+\\.$": {"type": "string"}
            }
          },
          "Outras produções bibliográficas": {
            "type": "object",
            "patternProperties": {
              "^[0-9]+\\.$": {"type": "string"}
            }
          },
          "Entrevistas, mesas redondas, programas e comentários na mídia": {
            "type": "object",
            "patternProperties": {
              "^[0-9]+\\.$": {"type": "string"}
            }
          },
          "Demais tipos de produção técnica": {
            "type": "object",
            "patternProperties": {
              "^[0-9]+\\.$": {"type": "string"}
            }
          }
        },
        "required": [
          "Artigos completos publicados em periódicos", 
          # "Apresentações de Trabalho",
          # 'Outras produções bibliográficas', 
          # "Entrevistas, mesas redondas, programas e comentários na mídia", 
          # "Demais tipos de produção técnica", 
          # "Resumos publicados em anais de congressos"
          ]
      },
      "ProjetosPesquisa": {
        "type": "array",
        "items": {
          "type": "object",
          "properties": {
            "chave": {"type": "string"},
            "titulo_projeto": {"type": "string"},
            "descricao": {"type": "string"}
          },
          "required": ["chave", "titulo_projeto", "descricao"]
        }
      },
      "ProjetosExtensão": {"type": "array"},
      "ProjetosDesenvolvimento": {
        "type": "array",
        "items": {
          "type": "object",
          "properties": {
            "chave": {"type": "string"},
            "titulo_projeto": {"type": "string"},
            "descricao": {"type": "string"}
          },
          "required": ["chave", "titulo_projeto", "descricao"]
        }
      },
      "ProjetosOutros": {"type": "array"},
      "Patentes e registros": {"type": "object"},
      "Bancas": {
        "type": "object",
        "properties": {
          "Participação em bancas de trabalhos de conclusão": {
            "type": "object",
            "patternProperties": {
              "^[0-9]+\\.$": { "type": "string" }
            }
          },
          "Participação em bancas de comissões julgadoras": {
            "type": "object",
            "patternProperties": {
              "^[0-9]+\\.$": { "type": "string" }
            }
          }
        },
        "required": [
          # "Participação em bancas de trabalhos de conclusão",
          # "Participação em bancas de comissões julgadoras"
        ]
      },
      "Orientações": {"type": "array"},
      "JCR2": {
        "type": "array",
        "items": {
          "type": "object",
          "properties": {
            "doi": { "type": ["string", "null"] },
            "impact-factor": {"type": "string"},
            "original_title": {"type": "string"}
          },
          "required": [
            # "doi", 
            # "impact-factor", 
            # "original_title"
            ]
        }
      }
    },
    "required": [
      "Identificação", 
      "Idiomas", 
      "Formação", 
      "Atuação Profissional", 
      "Linhas de Pesquisa", 
      "Áreas", 
      "Produções", 
      "ProjetosPesquisa", 
      "ProjetosExtensão", 
      "ProjetosDesenvolvimento", 
      "ProjetosOutros", 
      "Patentes e registros", 
      "Bancas", 
      "Orientações", 
      "JCR2",
      ]
  }
}

# Salvar o esquema em um arquivo JSON
with open('schema_curriculos.json', 'w') as f:
    json.dump(schema_curriculos, f, indent=4)

# Schema para dados sobre processos produtivos
schema_process = {
  "type": "object",
  "properties": {
    "nodes": {
      "type": "array",
      "items": {
        "type": "object",
        "properties": {
          "id": { "type": "string" },
          "label": { "type": "string" }
        },
        "required": ["id", "label"]
      }
    },
    "edges": {
      "type": "array",
      "items": {
        "type": "object",
        "properties": {
          "from": { "type": "string" },
          "to": { "type": "string" }
        },
        "required": ["from", "to"]
      }
    }
  },
  "required": ["nodes", "edges"]
}   

# Salvar o esquema em um arquivo JSON
filename = 'schema_process.json'
pathfilename = os.path.join(root_folder,'_data','in_json',filename)
with open(pathfilename, 'w') as f:
    json.dump(schema_process, f, indent=4)

In [ ]:
import os
from git import Repo
from graph_knowledge import GraphKnowledge as kg

repo = Repo(search_parent_directories=True)
root_folder = repo.working_tree_dir
filename = "levantamento_interesses.xlsx"

input_path = os.path.join(root_folder, "_data","in_xls")
output_path = os.path.join(root_folder, "_data","in_json")
kg.generate_interesses_json(os.path.join(input_path,filename), os.path.join(output_path,"input_interesses_pesquisadores.json"))

### Validar arquivos de dados de entrada

In [ ]:
validator = JSONValidator()

filenames = [
    'input_curriculos.json',
    'input_process_biologics.json',
    'input_process_smallmolecules.json',
    'input_gestao.json',
    ]

for filename in filenames:
    validator.validate_input(filename)

In [ ]:
# Extrair competências dos arquivos JSON
graph_data = []
for json_file in json_files:
    graph_data.extend(validator.extract_competencias(json_file))

Adaptar o código de extração de nós e arestas na função construct_graph de acordo com a estrutura dos seus arquivos JSON.

## Classe para criar o dataset do PyTorch:

- A classe GraphConstructor recebe uma lista de arquivos JSON como entrada e constrói o grafo de conhecimento a partir dos dados extraídos.
- A classe GraphDataset encapsula o grafo de conhecimento em um objeto Dataset do PyTorch, permitindo que ele seja usado no DataLoader da classe ModelPerformance.

A seguir instanciamos as classes para gerar o dataset.

In [ ]:
import json
from torch_geometric.data import Data
from torch.utils.data import Dataset

class GraphDataset(Dataset):
    def __init__(self, graph_data):
        self.graph_data = graph_data

    def __len__(self):
        return 1  # Retorna 1 pois temos apenas um grafo

    def __getitem__(self, idx):
        return self.graph_data

class GraphConstructor:
    def __init__(self, json_files):
        self.json_files = json_files
        self.graph_data = None

    def construct_graph(self):
        nodes = []
        edges = []
        node_idx = 0

        for json_file in self.json_files:
            with open(json_file, 'r') as f:
                data = json.load(f)

                # Extrair nós e arestas do JSON
                for node_data in data['nodes']:
                    nodes.append(node_data['features'])  # Assumindo que 'features' contém as features do nó
                    node_idx += 1

                for edge_data in data['edges']:
                    source = edge_data['source']
                    target = edge_data['target']
                    edges.append([source, target])

        # Criar o objeto Data do PyTorch Geometric
        x = torch.tensor(nodes, dtype=torch.float)
        edge_index = torch.tensor(edges, dtype=torch.long).t().contiguous()
        self.graph_data = Data(x=x, edge_index=edge_index)

        return self.graph_data        

In [ ]:
# Lista de arquivos JSON
json_files = ['competencias.json', 'processos.json', 'gestao.json']

# Criar o grafo de conhecimento
graph_constructor = GraphConstructor(json_files)
graph_data = graph_constructor.construct_graph()

# Criar o dataset do PyTorch
dataset = GraphDataset(graph_data)

# ... usar o dataset na classe ModelPerformance ...

## Função de perda para aprendizado não-supervisionado

No contexto de aprendizado não-supervisionado para alinhar competências da ICT aos produtos estratégicos, a escolha da função de perda considerou o objetivo de agrupar pesquisadores com competências semelhantes e identificar as lacunas em relação aos produtos estratégicos. Neste caso, não existe nesse cenário rótulos de classe pré-definidos. Portanto, as funções de perda tradicionais, como MSE ou Cross-Entropy, não são as mais adequadas.

Considerando as opções de abordagem em redes neurais (com KANs, com Fourier e Híbrida) que serão utilizadas nos problemas de aprendizado não-supervisionado, a **Triplet Loss** surge como uma escolha promissora para esse tipo de problema, pois se concentra em aprender embeddings que agrupam nós semelhantes e separam nós distintos, o que se alinha com o objetivo de agrupar pesquisadores com competências semelhantes e identificar lacunas.

O raciocínio sobre a Triplet Loss ser uma boa escolha para o aprendizado não-supervisionado em grafos com o objetivo de alinhar competências se aplica não só para abordagem com KANs mas também, de forma similar, se aplica bem, com algumas adaptações, para as outras abordagens de Rede Neural com Fourier e Rede Neural Híbrida.

### Como aplicar a Triplet Loss

1. **Definir trios:**
    * **Âncora:** Um pesquisador.
    * **Positivo:** Outro pesquisador com competências semelhantes à âncora, idealmente trabalhando em um produto estratégico que requer essas competências.
    * **Negativo:** Um pesquisador com competências diferentes da âncora, preferencialmente que não esteja associado a nenhum produto estratégico que a âncora poderia contribuir.

2. **Calcular perda:**
    * A Triplet Loss busca minimizar a distância entre a âncora e o positivo, e maximizar a distância entre a âncora e o negativo.
    * A função de perda é definida como:
      
<center>L = max(d(âncora, positivo) - d(âncora, negativo) + margem, 0)</center>
      
<center>onde 'd(a, b)' é a distância entre os embeddings de 'a' e 'b', e 'margem' é um hiperparâmetro que define a diferença mínima desejada entre as distâncias.</center>

3. **Amostrar trios:**
    * A escolha dos trios é crucial para o bom desempenho da Triplet Loss.
    * Estratégias de amostragem como "hard negative mining" podem ser utilizadas para selecionar os trios mais informativos, que contribuem mais para o aprendizado.

### Vantagens da Triplet Loss

* **Alinhamento com o objetivo:** A Triplet Loss foca em agrupar nós semelhantes e separar nós distintos, o que é diretamente relevante para o problema de alinhamento de competências.
* **Consideração da estrutura do grafo:** A escolha dos trios pode levar em conta as relações entre os nós no grafo, como a colaboração entre pesquisadores ou a participação em projetos comuns.
* **Flexibilidade:** A Triplet Loss pode ser combinada com diferentes arquiteturas de redes neurais em grafos, como GNNs, para capturar as informações do grafo de forma mais eficiente.


### Implementação da função de perda com Triplet Loss

Para implementar a Triplet Loss, foi utilizada biblioteca `PyTorch Metric Learning`, que oferece diversas funções de perda e métodos de mineração para facilitar o treinamento.

Lembre-se que a escolha da função de perda é apenas um dos aspectos do problema. A arquitetura da rede neural, a estratégia de amostragem dos trios e a escolha dos hiperparâmetros também são importantes para o bom desempenho do modelo.

# Tabelas LaTeX e mapas mentais

In [ ]:
# Crie uma instância da classe, passando o arquivo JSON e o arquivo BibTeX


mindmap = MindmapGenerator('data.json', 'referencias.bib')

# Gere o mapa mental
mindmap.generate_mindmap('mindmap.png')

# Gere a tabela LaTeX com as contagens de tipos de entrada do arquivo BibTeX
mindmap.generate_latex_table()

# <b>Abordagens GNN com função perda Triplet Loss</b>

## Rede KAN com função de perda por Triplet Loss:

A KAN foi responsável por gerar os embeddings dos nós do grafo, que foram utilizados pela Triplet Loss para calcular as distâncias entre os nós. A saída da KAN é um vetor de embedding para cada nó, representando suas características e relações no grafo.

### Definição dos trios:

A definição dos trios (âncora, positivo e negativo) segue a mesma lógica das outras abordagens:
- Âncora: Um pesquisador.
- Positivo: Outro pesquisador com competências semelhantes à âncora, idealmente associado a um produto estratégico que demanda essas competências.
- Negativo: Um pesquisador com competências diferentes da âncora, preferencialmente não associado a produtos estratégicos que a âncora poderia contribuir.


### Cálculo da perda:

- A Triplet Loss foi calculada com base nos embeddings gerados pela Kolmogorov-Arnold Networks (KAN). 

- Quanto à amostragem dos trios, foi utilizada uma estratégia de amostragem eficiente, a "hard negative mining", para selecionar os trios mais informativos para o aprendizado.

- A Triplet Loss foi utilizada para aprender embeddings que representem tanto as características estruturais do grafo quanto as relações de similaridade entre os nós, porém com o diferencial de aprender sobre as relações que ligam os nós (arestas).

- Quanto à otimização, durante o treinamento, o otimizador ajustará os pesos da KAN para minimizar a Triplet Loss. A função de perda buscou minimizar a distância entre a âncora e o positivo, e maximizar a distância entre a âncora e o negativo. Isso gerou embeddings que agrupam pesquisadores com competências semelhantes e separam aqueles com competências distintas.

## Rede Neural Fourier (Transformadas de Fourier + GNN):

* A Triplet Loss pode ser utilizada em conjunto com as Transformadas de Fourier para aprender embeddings que representem tanto as características estruturais do grafo quanto as relações de similaridade entre os nós.

* As Transformadas de Fourier podem auxiliar na identificação de padrões e características relevantes na estrutura do grafo,  enquanto a Triplet Loss guia o aprendizado para agrupar nós com competências semelhantes.

* A combinação dessas técnicas pode levar a um modelo mais robusto e capaz de capturar diferentes aspectos do problema de alinhamento de competências.

## Rede Neural Híbrida (Controle de Sincronização + GNN):

* A Triplet Loss foi utilizada para auxiliar na sincronização dos nós, incentivando o agrupamento de pesquisadores com competências semelhantes e a separação daqueles com competências diferentes.

* A dinâmica de sincronização, baseada no valor de Fiedler, pode complementar a Triplet Loss, ajudando a identificar os nós mais importantes para o alinhamento de competências e guiando o aprendizado da rede.

* A combinação da Triplet Loss com a GNN permite capturar as informações locais e globais do grafo,  enquanto a dinâmica de sincronização fornece uma perspectiva adicional sobre a importância dos nós e arestas.

**Observações Gerais:**

A escolha da função de perda é apenas um dos aspectos do problema. A arquitetura da rede neural, a estratégia de amostragem dos trios e a escolha dos hiperparâmetros também são importantes para o bom desempenho do modelo. Outras funções de perda, como a DGI ou a VGAE, também podem ser exploradas, especialmente se você desejar capturar a estrutura global do grafo e aprender representações mais robustas. É crucial avaliar o desempenho do modelo com diferentes métricas de avaliação, como as discutidas anteriormente, para garantir que ele esteja alinhando as competências da ICT aos produtos estratégicos de forma eficaz.

A Triplet Loss é uma função de perda versátil que pode ser aplicada em diferentes abordagens de redes neurais em grafos. Para o problema de alinhamento de competências em aprendizado não-supervisionado, a escolha da melhor função de perda e da arquitetura do modelo dependerá das características específicas do seu problema e dos seus objetivos, mas pode-se observar de forma geral, que:

* A escolha dos trios (âncora, positivo e negativo) é crucial para o bom desempenho da Triplet Loss. É importante definir uma estratégia de amostragem que leve em conta as características do grafo e o objetivo de alinhamento de competências.
* A margem da Triplet Loss é um hiperparâmetro importante que deve ser ajustado para obter o melhor desempenho.
* A Triplet Loss pode ser combinada com outras funções de perda, como a DGI ou a VGAE, para capturar diferentes aspectos do problema e melhorar o aprendizado das representações.
* É fundamental avaliar o desempenho do modelo com diferentes métricas de avaliação,  além de analisar os embeddings e clusters gerados para garantir que o modelo esteja alinhando as competências de forma eficaz e interpretável.


A interpretabilidade do modelo também é importante. Analise os embeddings gerados e os clusters formados para entender como o modelo está realizando o alinhamento de competências.
Ao combinar a Triplet Loss com uma arquitetura de rede neural em grafos adequada e uma estratégia de amostragem eficiente, você poderá desenvolver um modelo de aprendizado não-supervisionado capaz de alinhar as competências da ICT aos produtos estratégicos e auxiliar na identificação de lacunas e oportunidades de desenvolvimento.

## Parâmetros iniciais e ajuste de hiperparâmetros

A escolha dos parâmetros para cada modelo depende de diversos fatores, como o tamanho e a complexidade do grafo de conhecimento, a quantidade de dados de treinamento, a capacidade computacional disponível e o objetivo da análise. 

Partimos de alguns valores iniciais razoáveis para cada parâmetro para ajustá-los posteriormente com base nos resultados dos experimentos.

* `num_features`: Depende do número de features para cada nó do grafo (pesquisadores, competências e produtos). 
    * **Exemplo:** Se você tiver 10 features para cada pesquisador (e.g., anos de experiência, número de publicações, áreas de atuação), 5 features para cada competência (e.g., nível de proficiência, tipo de competência) e 3 features para cada produto estratégico (e.g., complexidade, área terapêutica), você pode concatenar esses features em um vetor de 18 dimensões.
    * **Valor inicial:**  Número total de features extraídas para cada nó.

* `hidden_dim`: Define a dimensão das camadas ocultas da KAN.
    * **Valor inicial:**  64 ou 128.  Experimentar variações em potências de 2.

* `num_classes`: Define o número de classes (clusters) que você deseja gerar.
    * **Valor inicial:**  Mesmo valor utilizado no modelo híbrido.

* `num_layers`: Define o número de camadas na KAN.
    * **Valor inicial:**  3 ou 4 é um bom início. Escolher um valor que faça sentido para o problema.  Você pode usar técnicas como o método do cotovelo para auxiliar na escolha do número ideal de clusters.

* `dropout`: Define a taxa de dropout.
    * **Valor inicial:**  0.5.

### Observações:

* **Ajuste dos parâmetros:**  É fundamental realizar experimentos e analisar os resultados para ajustar os parâmetros e encontrar a melhor configuração para cada modelo.
* **Técnicas de otimização:**  Utilize técnicas como grid search ou random search para explorar diferentes combinações de parâmetros e encontrar a configuração ótima.
* **Métricas de avaliação:**  Monitore as métricas de avaliação durante o treinamento e a validação para avaliar o desempenho do modelo e guiar o ajuste dos parâmetros.

Estes foram apenas os valores iniciais. A melhor configuração para os parâmetros dependerá das características do grafo de conhecimento, da quantidade de dados de treinamento e do objetivo da análise. Esses valores foram buscados por otimização automatizada de hiperparâmetros dentro de espaços de busca relacionados com os valores inciais mostrados acima. A partir dos desse ajuste e de vários experimentos para medição de desempenho computacional é que se chegou aos hiperparâmetros adequados para obter o melhor desempenho de cada modelo.

# <b>Visualização dos dados do Dataset</b>

Para criar a estrutura de manipulação dos arquivos JSON e construir o grafo de análise, primeiro carregamos os dados dos arquivos JSON e, em seguida, criamos o grafo de conhecimento.

In [ ]:
from graph_knowledge import GraphKnowledge as kg
kg.info_dataset()

In [19]:
# %pip install pytorch-metric-learning

# Importações necessárias
import os
import json
import cudf
import cugraph
import warnings
import numpy as np
import pandas as pd
warnings.simplefilter(action='ignore', category=FutureWarning)

from git import Repo
from pyvis.network import Network
from IPython.display import HTML, display
from semantic_matcher import RedeNeuralHibrida, RedeNeuralKAN, RedeNeuralFourier

def display_full_width_df(df):
    """
    Formats the DataFrame to occupy the entire width of the cell, uses line breaks, and justifies the text to the left.

    Args:
        df (pd.DataFrame): The DataFrame to be formatted.
    """

    styled_df = df.to_html(classes='full-width-df')
    css_style = """
    <style>
    .full-width-df td {
        word-wrap: break-word;
        white-space: pre-wrap;
        text-align: left;
    }
    </style>
    """
    display(HTML(css_style + styled_df))

    # .full-width-df {
    #      width: 100%;
    #  }
    # return css_style + styled_df

def recuperar_questionario():
    filename = 'Alinhamento de competências em pesquisa às políticas para desenvolvimento do CEIS.xlsx'
    pathfilename = os.path.join(xlsx_folder, filename)    
    df_questionario = pd.read_excel(pathfilename).iloc[:, 7:]
    print('Questionamentos realizados em levantamento junto aos pesquisadores da ICT:')
    for n,i in enumerate(df_questionario.keys()):
        if n==0:
            print('\nQuestões e palavras-chave de pesquisa:')
        elif n==2:
            print('\nCompetências já dominadas e a evoluir:')
        elif n==4:
            print('\nIntenção de atuar em atividades de desenvolvimento tencológico:')
        elif n==5:
            print('\nNível de maturidade segmentado por tipo de tecnologias:')
        elif n==11:
            print('\nConhecimentos sobre Estratégia para Desenvolvimento do CEIS, quanto:')
        elif n==15:
            print('\nProdutos mais próximos das pesquisas na ICT:')
        elif n==19:
            print('\nOcupação do tempo de trabalho por tipos de atividades de trabalho:')
        elif n==25:
            print('\nSatisfação e identificação:')        
        print(f"  {n+1:2} {i}")
    return df_questionario

def montar_dataframes_levantamento():
    filename = 'Alinhamento de competências em pesquisa às políticas para desenvolvimento do CEIS.xlsx'
    pathfilename = os.path.join(xlsx_folder, filename)
    df_levantamento = pd.read_excel(pathfilename).iloc[:, 7:]
    df_levantamento.columns=['questoes_pesquisa','palavras_chave','compet_dominadas','compet_desenvolver','avancar_desenv_desafios',
                            'tec_diagnostico','tec_pesquisa','tec_terapeutica','tec_serviço','tec_social','tec_digital',
                            'conhec_blocos','conhec_desafios','conhec_plataformas','conhec_produtos',
                            'desafio_mais_proximo','produtos_bloco1','produtos_bloco2','adicional_contibuir',
                            'tempo_lev_dados','tempo_analise_dados','tempo_debates_grupo','tempo_redigir_textos','tempo_reunioes','tempo_comunicacoes',                      
                            'satisfacao','nome']

    # df_levantamento.drop(columns=['tempo_fx1','tempo_fx2','tempo_fx3','tempo_fx4','tempo_fx5','tempo_fx6'], inplace=True)
    print('Questionamentos realizados em levantamento junto aos pesquisadores da ICT:')
    for n,i in enumerate(df_levantamento.keys()):
        if n==0:
            print('\nQuestões e palavras-chave de pesquisa:')
        elif n==2:
            print('\nCompetências já dominadas e a evoluir:')
        elif n==4:
            print('\nIntenção de atuar em atividades de desenvolvimento tencológico:')
        elif n==5:
            print('\nNível de maturidade segmentado por tipo de tecnologias:')
        elif n==11:
            print('\nConhecimentos sobre Estratégia para Desenvolvimento do CEIS, quanto:')
        elif n==15:
            print('\nProdutos mais próximos das pesquisas na ICT:')
        elif n==19:
            print('\nOcupação do tempo de trabalho por tipos de atividades de trabalho:')
        elif n==25:
            print('\nSatisfação e identificação:')        
        print(f"  {n+1:2} {i}")

    # Criar df_questoes com as 5 primeiras colunas do df_levantamento
    df_questoes = df_levantamento.iloc[:, :5]

    # Adicionar coluna 'id_resposta' ao df_questoes 
    df_questoes['id_resposta'] = df_levantamento.index

    # Adicionar coluna 'nome_pesquisador' ao df_questoes com os valores da última coluna do df_levantamento 
    # df_questoes['nome_pesquisador'] = df_levantamento.iloc[:, -1]

    # Montar dataframe com faixas de TRL por cada tipo de tecnologia
    df_trls = df_levantamento.iloc[:,5:11]
    df_trls['id_resposta'] = df_levantamento.index

    # Montar dataframe sobre o conhecimento atual sobre estratégia do CEIS
    df_estrat_ceis = df_levantamento.iloc[:,11:15]
    df_estrat_ceis['id_resposta'] = df_levantamento.index

    # Montar dataframe sobre os desafios e produtos mais próximos dentro da estratégia do CEIS
    df_produtos_proximos = df_levantamento.iloc[:,15:19]
    df_produtos_proximos['id_resposta'] = df_levantamento.index

    # Montar dataframe sobre tempo destinado aos tipos de atividades típicas em pesquisa
    df_tempo_ativ = df_levantamento.iloc[:,19:25]
    df_tempo_ativ['id_resposta'] = df_levantamento.index

    # Montar dataframe sobre satisfação com a coordenação de pesquisa e nome
    df_satisf = df_levantamento.iloc[:,25:-1]

    return df_levantamento, df_questoes, df_trls, df_estrat_ceis, df_produtos_proximos, df_tempo_ativ, df_satisf

def listar_questoes_pesquisa(df_questoes):
    print('Questões de pesquisa:')
    pular=[np.nan,'','as principais questões científicas que norteiam minhas pesquisas na fiocruz ceará, relacionadas ao enfrentamento dos desafios em saúde, envolvem:','essas questões são centrais para o desenvolvimento de soluções que melhorem a vigilância epidemiológica e o acesso ao diagnóstico no brasil.']
    for i in df_questoes.iloc[:,0]:
        try:
            if i.strip().lower() not in pular:
                i = i.replace(';','\n')
                for sub_i in i.split('\n'):
                    if sub_i.strip().lower() not in pular:
                        print(f'  {sub_i.strip().lower()}')
        except:
            pass

def listar_palavras_chave(df_questoes):
    print('Palavras-Chave:')
    pular=['','as principais palavras-chave que podem associar meus temas de pesquisa com oportunidades de fomento que desejo monitorar são:']
    for i in df_questoes.iloc[:,1]:
        try:
            if i.strip().lower() not in pular:
                i = i.replace(';','\n')
                for sub_i in i.split('\n'):
                    if sub_i.strip().lower() not in pular:
                        print(f'  {sub_i.strip().lower()}')
        except:
            pass

def listar_desafios_proximos(df_produtos_proximos):
    print('Desafios mais próximos das pesquisas na ICT:')
    pular = ['', ' ', 'não encontrei produto algum na lista acima que corresponda com minhas pesquisas atuais e no futuro próximo']
    for i in df_produtos_proximos.iloc[:,0]:
        try:
            # Verifica se a string 'i' não está vazia e não contém apenas espaços em branco
            if i.strip() and i.lower().strip() not in pular:
                i = i.replace(';','\n')
                for sub_i in i.split('\n'):
                    if sub_i.strip().lower() not in pular:
                        print(f'  {sub_i.strip().lower()}')
            print()
        except:
            pass

    # Divide a coluna de desafios em múltiplas colunas, separando os desafios por ponto e vírgula
    df_produtos_proximos.iloc[:, 0] = df_produtos_proximos.iloc[:, 0].str.split(';')

    # "Explode" o DataFrame para que cada desafio fique em uma linha separada
    df_exploded = df_produtos_proximos.explode(df_produtos_proximos.columns[0])

    # Filtra os valores vazios ou com apenas espaços em branco
    df_exploded = df_exploded[df_exploded.iloc[:, 0].str.strip().astype(bool)]

    # Calcula a frequência de cada desafio
    df_sum = df_exploded[df_produtos_proximos.columns[0]].value_counts()

    # Converte a Series para DataFrame e redefine o índice
    df = pd.DataFrame(df_sum).reset_index()

    # Renomeia as colunas para 'Desafio' e 'Total'
    df.columns = ['Desafio', 'Total']
    df_full = display_full_width_df(df)

def listar_produtos_proximos_emergencias(df_produtos_proximos):
    print('Produtos do Bloco 01 (Preparação do Sistema de Saúde para Emergências Sanitárias) mais próximos das pesquisas na ICT:')
    pular = ['', 'não encontrei produto algum na lista acima que corresponda com minhas pesquisas atuais e no futuro próximo']
    for i in df_produtos_proximos.iloc[:,1]:
        try:
            if i.lower().strip() not in pular:
                i = i.replace(';','\n')
                for sub_i in i.split('\n'):
                    if sub_i.strip().lower() not in pular:
                        print(f'  {sub_i.strip().lower()}')
        except:
            pass   

def listar_produtos_proximos_agravos(df_produtos_proximos):
    print('Produtos do Bloco 02 (Doenças e Agravos Críticos para o SUS) mais próximos das pesquisas na ICT:')
    pular = ['','não encontrei agravo algum na lista acima que corresponda com minhas pesquisas atuais e no futuro próximo']
    for i in df_produtos_proximos.iloc[:,2]:
        try:
            if i.strip().lower() not in pular:
                i = i.replace(';','\n')
                for sub_i in i.split('\n'):
                    if sub_i.strip().lower() not in pular:
                        print(f'  {sub_i.strip().lower()}')
        except:
            pass

def produtos_extra(df_produtos_proximos):
    print('Produtos extra estratégia do CEIS:')
    pular = [np.nan,'não sei responder.','não consigo enxergar','já associado']
    for i in df_produtos_proximos.iloc[:,3]:
        try:
            if i.strip().lower() not in pular:
                i = i.replace(';','\n')
                for sub_i in i.split('\n'):
                    if sub_i.strip().lower() not in pular:
                        print(f'  {sub_i.strip().lower()}')
        except:
            pass

def df_to_json(dataframe):
  # Converte o DataFrame para JSON no formato 'records' com 'lines=True' e UTF-8
  json_data = dataframe.to_json(orient='records', lines=True, force_ascii=False)

  # Divide a string JSON em uma lista de strings, uma para cada linha
  json_lines = json_data.splitlines()

  # Converte cada linha em um objeto JSON
  json_objects = []
  for line in json_lines:
    try:
      json_objects.append(json.loads(line))
    except json.JSONDecodeError as e:
      print(f"Erro ao analisar a linha: {line}")
      print(e)

  # Imprime a lista de objetos JSON
  return json_objects

## Dados de levantamento de Questões e Palavras-Chave

In [ ]:
df_levantamento, df_questoes, df_trls, df_estrat_ceis, df_produtos_proximos, df_tempo_ativ, df_satisf = montar_dataframes_levantamento()

In [ ]:
df_questionario = recuperar_questionario()
# df_questionario.keys()

In [ ]:
df_questoes

In [ ]:
json_questoes = df_to_json(df_questoes)
json_questoes

In [ ]:
listar_questoes_pesquisa(df_questoes)

In [ ]:
listar_palavras_chave(df_questoes)

## Alinhamento com a estratégia de desenvolvimento do CEIS

In [ ]:
listar_desafios_proximos(df_produtos_proximos)

In [ ]:
listar_produtos_proximos_emergencias(df_produtos_proximos)

In [ ]:
json_produtos_proximos = df_to_json(df_produtos_proximos)
json_produtos_proximos

In [ ]:
listar_produtos_proximos_agravos(df_produtos_proximos)

In [ ]:
produtos_extra(df_produtos_proximos)

In [ ]:
df_trls

In [ ]:
df_estrat_ceis

In [ ]:
df_produtos_proximos

In [ ]:
df_tempo_ativ

In [ ]:
df_satisf

In [22]:
# print("Lista de relacionamentos por tipo de produtos")
# for i,j in relacoes_biologicos.items():
#     for k in j:
#         print(k.keys())

## Mapas de Processos de Biológicos e Pequenas Moléculas

In [ ]:
print("Lista de relacionamentos por tipo de produtos")
for i,j in relacoes_pequenas_moleculas.items():
    for k in j:
        rotulo = k.get('id')
        if rotulo:
            print(f"  {rotulo}")
        else:
            print(f"{  k}")

In [ ]:
print("Lista de relacionamentos por tipo de produtos")
for i,j in relacoes_biologicos.items():
    for k in j:
        rotulo = k.get('id')
        if rotulo:
            print(f"  {rotulo}")
        else:
            print(f"{  k}")

# <b>Preparar o Grafo de Conhecimento</b>


In [ ]:
import networkx as nx

grafo = nx.Graph()
# ... (código para adicionar nós e arestas ao grafo)

grafo_treino, grafo_teste = train_test_split(grafo, test_size=0.2)

# Configuração dos parâmetros dos modelos
parametros_iniciais = {
    'num_features': ...,  # Número de features dos nós
    'hidden_dim': 4,  # Dimensão da camada oculta
    'num_classes': 7,  # Número de classes (clusters)
    'dropout': 0.5  # Taxa de dropout
}

# 3. Instanciar classes de cada modelo

In [ ]:
modelo_kan     = RedeNeuralKAN(parametros_iniciais)
modelo_fourier = RedeNeuralFourier(parametros_iniciais)
modelo_hibrido = RedeNeuralHibrida(parametros_iniciais)

# 4. Treinar os modelos

Utilizar o grafo de treinamento (grafo_treino) para treinar cada um dos modelos. 

## a) Treinamento do modelo de GNN com Sincronização (híbrido)

    Inspirado em https://iopscience.iop.org/article/10.1209/0295-5075/ad76d6

In [5]:
def treinar_modelo(modelo, grafo_treino, epochs=100, learning_rate=0.01):
    """
    Treina um modelo de rede neural em grafos.

    Args:
      modelo: O modelo da rede neural.
      grafo_treino: O grafo de conhecimento para treinamento.
      epochs: O número de épocas de treinamento.
      learning_rate: A taxa de aprendizado do otimizador.

    Funcionamento:
      Recebe o modelo, o grafo de treinamento, o número de épocas e a taxa de aprendizado como parâmetros.
      Cria um otimizador Adam para atualizar os pesos do modelo.
      Define a função de perda como CrossEntropyLoss (ajustável para a função de perda mais adequada ao problema).
      Prepara os dados de treinamento (features, edge_index, edge_weight e labels, se houver).
      Itera pelas épocas de treinamento, executando o modelo, calculando a perda, os gradientes e atualizando os pesos.
    """
    try:
        # Criar o otimizador
        optimizer = torch.optim.Adam(modelo.parameters(), lr=learning_rate)

        # Definir a função de perda (exemplo: cross-entropy)
        criterion = nn.CrossEntropyLoss()

        # Preparar os dados de treinamento
        x = torch.tensor(list(nx.get_node_attributes(grafo_treino, 'features').values()), dtype=torch.float)
        edge_index = torch.tensor(list(grafo_treino.edges()), dtype=torch.long).t().contiguous()
        edge_weight = torch.tensor(list(nx.get_edge_attributes(grafo_treino, 'weight').values()), dtype=torch.float)
        
        # ... (código para obter os rótulos dos nós, se houver)
        # labels = ...

        # Treinar o modelo
        modelo.train()  # Mudar para o modo de treinamento
        for epoch in range(epochs):
            optimizer.zero_grad()  # Zerar os gradientes
            out = modelo(x, edge_index, edge_weight)  # Executar o modelo
            loss = criterion(out, labels)  # Calcular a perda
            loss.backward()  # Calcular os gradientes
            optimizer.step()  # Atualizar os pesos do modelo

            print(f'Epoch {epoch+1}/{epochs}, Loss: {loss.item()}')

    except Exception as e:
        print(f"Erro no treinamento do modelo: {e}")
        raise e

# Chamar a função para treinar o modelo_hibrido
treinar_modelo(modelo_hibrido, grafo_treino, epochs=100, learning_rate=0.01)

## b) Função de treinamento do modelo_kan_mse

    Com função de perda Mean Squared Error

In [ ]:
criterion_kan = nn.MSELoss()  

def treinar_modelo_kan_mse(modelo, grafo_treino, epochs=100, learning_rate=0.01, criterion=criterion_kan):
    """
    Treina um modelo de rede neural KAN em grafos.

    Args:
      modelo: O modelo da rede neural KAN.
      grafo_treino: O grafo de conhecimento para treinamento.
      epochs: O número de épocas de treinamento.
      learning_rate: A taxa de aprendizado do otimizador.
      criterion: A função de perda.
    
    Função de Perda: A função de perda utilizada foi o MSELoss. 
    
    Pode-se ajustar para uma função de perda mais adequada ao problema, considerando aprendizado não-supervisionado. 
    Uma opção seria utilizar uma função de perda baseada na distância entre os embeddings de nós que deveriam estar no mesmo cluster.
    Ou ainda, pode-se também usar função de perda baseada em Triplet Loss.
    
    Rótulos: Ajustar o código para obter os rótulos dos nós (labels) de acordo com o seu problema. 
    Como o problema é de aprendizado não-supervisionado, os rótulos podem ser definidos com base em alguma heurística ou conhecimento prévio sobre o problema.
    
    Hiperparâmetros: Ajustar o número de épocas (epochs) e a taxa de aprendizado (learning_rate) para obter o melhor desempenho do modelo.    
    """
    try:
        # Criar o otimizador
        optimizer = torch.optim.Adam(modelo.parameters(), lr=learning_rate)

        # Preparar os dados de treinamento
        x = torch.tensor(list(nx.get_node_attributes(grafo_treino, 'features').values()), dtype=torch.float)
        edge_index = torch.tensor(list(grafo_treino.edges()), dtype=torch.long).t().contiguous()
        edge_weight = torch.tensor(list(nx.get_edge_attributes(grafo_treino, 'weight').values()), dtype=torch.float)
        
        # ... (código para obter os rótulos dos nós, se houver - ajustar para o seu problema)
        # labels = ...

        # Treinar o modelo
        modelo.train()  # Mudar para o modo de treinamento
        for epoch in range(epochs):
            optimizer.zero_grad()  # Zerar os gradientes
            out = modelo(x, edge_index, edge_weight)  # Executar o modelo
            
            # Calcular a perda (considerando que labels é um tensor com os valores desejados para os nós)
            loss = criterion(out, labels)  
            loss.backward()  # Calcular os gradientes
            optimizer.step()  # Atualizar os pesos do modelo

            print(f'Epoch {epoch+1}/{epochs}, Loss: {loss.item()}')

    except Exception as e:
        print(f"Erro no treinamento do modelo KAN: {e}")
        raise e

# Chamar a função para treinar o modelo_kan
treinar_modelo_kan_mse(modelo_kan, grafo_treino, epochs=100, learning_rate=0.01)

## c) Função de treinamento do modelo_kan_triplet

    Com função de perda por triplet loss

In [ ]:
from pytorch_metric_learning import losses

# Exemplo de função para gerar trios (âncora, positivo, negativo)
def gerar_triplets(grafo, embeddings):
    """
    Gera trios (âncora, positivo, negativo) para a Triplet Loss.

    Args:
      grafo: O grafo de conhecimento.
      embeddings: Os embeddings dos nós.

    Returns:
      Um tensor com os trios.
    """
    triplets = []
    for no_ancora in grafo.nodes():
        # Encontrar nós positivos (similares à âncora)
        # ... (implementar lógica para encontrar nós positivos)

        # Encontrar nós negativos (diferentes da âncora)
        # ... (implementar lógica para encontrar nós negativos)

        for no_positivo in nos_positivos:
            for no_negativo in nos_negativos:
                triplets.append([no_ancora, no_positivo, no_negativo])

    return torch.tensor(triplets)

def treinar_modelo_kan_triplet(modelo, grafo_treino, epochs=100, learning_rate=0.01):
    """
    Treina um modelo de rede neural KAN em grafos utilizando Triplet Loss.

    Args:
      modelo: O modelo da rede neural KAN.
      grafo_treino: O grafo de conhecimento para treinamento.
      epochs: O número de épocas de treinamento.
      learning_rate: A taxa de aprendizado do otimizador.
    """
    try:
        # Criar o otimizador
        optimizer = torch.optim.Adam(modelo.parameters(), lr=learning_rate)

        # Preparar os dados de treinamento
        x = torch.tensor(list(nx.get_node_attributes(grafo_treino, 'features').values()), dtype=torch.float)
        edge_index = torch.tensor(list(grafo_treino.edges()), dtype=torch.long).t().contiguous()
        edge_weight = torch.tensor(list(nx.get_edge_attributes(grafo_treino, 'weight').values()), dtype=torch.float)

        # Criar a função de perda Triplet Loss
        loss_func = losses.TripletMarginLoss()

        # Treinar o modelo
        modelo.train()  # Mudar para o modo de treinamento
        for epoch in range(epochs):
            optimizer.zero_grad()  # Zerar os gradientes
            embeddings = modelo(x, edge_index, edge_weight)  # Executar o modelo
            
            # Gerar os trios (âncora, positivo, negativo), com lógica de acordo com o problema
            triplets = gerar_triplets(grafo_treino, embeddings) 

            # Calcular a Triplet Loss
            loss = loss_func(embeddings, triplets)  
            loss.backward()  # Calcular os gradientes
            optimizer.step()  # Atualizar os pesos do modelo

            print(f'Epoch {epoch+1}/{epochs}, Loss: {loss.item()}')

    except Exception as e:
        print(f"Erro no treinamento do modelo KAN: {e}")
        raise e

# Chamar a função para treinar o modelo_kan
treinar_modelo_kan_triplet(modelo_kan, grafo_treino, epochs=100, learning_rate=0.01)

## d) Função para treinamento do modelo_fourier

In [ ]:
# Treinar o modelo_fourier
criterion_fourier = nn.KLDivLoss() # Ajustar a função de perda para o modelo Fourier (exemplo: Kullback-Leibler Divergence)  

def treinar_modelo_fourier(modelo, grafo_treino, epochs=100, learning_rate=0.01, criterion=criterion_fourier):
    """
    Treina um modelo de rede neural Fourier em grafos.

    Args:
      modelo: O modelo da rede neural Fourier.
      grafo_treino: O grafo de conhecimento para treinamento.
      epochs: O número de épocas de treinamento.
      learning_rate: A taxa de aprendizado do otimizador.
      criterion: A função de perda.
    """
    try:
        # Criar o otimizador
        optimizer = torch.optim.Adam(modelo.parameters(), lr=learning_rate)

        # Preparar os dados de treinamento
        x = torch.tensor(list(nx.get_node_attributes(grafo_treino, 'features').values()), dtype=torch.float)
        edge_index = torch.tensor(list(grafo_treino.edges()), dtype=torch.long).t().contiguous()
        edge_weight = torch.tensor(list(nx.get_edge_attributes(grafo_treino, 'weight').values()), dtype=torch.float)
        # ... (código para obter os rótulos dos nós, se houver - ajustar para o seu problema)
        # labels = ...

        # Treinar o modelo
        modelo.train()  # Mudar para o modo de treinamento
        for epoch in range(epochs):
            optimizer.zero_grad()  # Zerar os gradientes
            out = modelo(x, edge_index, edge_weight)  # Executar o modelo

            # Ajustar a saída do modelo e os rótulos para a KLDivLoss
            # (a saída deve ser um tensor de probabilidades e os rótulos devem ser um tensor de índices)
            out = torch.nn.functional.log_softmax(out, dim=1)  # Aplicar log_softmax na saída
            # Converter labels para one-hot encoding e aplicar log_softmax (ajuste para o seu problema)
            labels_onehot = torch.nn.functional.one_hot(labels, num_classes=out.shape[1]).float()
            labels_onehot = torch.nn.functional.log_softmax(labels_onehot, dim=1) 

            # Calcular a perda 
            loss = criterion(out, labels_onehot)  
            loss.backward()  # Calcular os gradientes
            optimizer.step()  # Atualizar os pesos do modelo

            print(f'Epoch {epoch+1}/{epochs}, Loss: {loss.item()}')

    except Exception as e:
        print(f"Erro no treinamento do modelo Fourier: {e}")
        raise e

# Chamar a função para treinar o modelo_fourier
treinar_modelo_fourier(modelo_fourier, grafo_treino, epochs=100, learning_rate=0.01)

# 5. Instanciar a classe de testes para realizar análises:

In [ ]:
# Crie uma instância da classe TesteRedeNeural para avaliar o desempenho de cada modelo no grafo de teste (grafo_teste):
teste_rede_neural = TesteRedeNeural(grafo_teste, parametros_modelo)  # Instanciar a classe de testes

# Controle de Sincronização + GNN
clusters_hibrido, embeddings_hibrido = modelo_hibrido.inferir(grafo_teste)  # Realizar inferência
analises_hibrido = teste_rede_neural.testar_inferencia(clusters_hibrido, embeddings_hibrido)

# KAN + GNN
clusters_kan, embeddings_kan = modelo_kan.inferir(grafo_teste)
analises_kan = teste_rede_neural.testar_inferencia(clusters_kan, embeddings_kan)

# Transformadas de Fourier + GNN
clusters_fourier, embeddings_fourier = modelo_fourier.inferir(grafo_teste)
analises_fourier = teste_rede_neural.testar_inferencia(clusters_fourier, embeddings_fourier)

# 6. Comparação dos resultados
def gerar_tabela_latex(analises):
    """
    Gera uma tabela LaTeX com os resultados das análises.

    Args:
      analises: Um dicionário contendo as análises dos modelos.

    Returns:
      Uma string com o código LaTeX da tabela.
    """
    df = pd.DataFrame(analises)
    df.index = ['Híbrido', 'KAN', 'Fourier']
    latex_table = df.to_latex(float_format="%.3f", caption="Resultados das Análises", label="tab:resultados")
    return latex_table

# Agregar as análises em um dicionário
analises = {
    'Híbrido': analises_hibrido,
    'KAN': analises_kan,
    'Fourier': analises_fourier,
}

# Gerar a tabela LaTeX
tabela_latex = gerar_tabela_latex(analises)

# Exibir a tabela LaTeX
print(tabela_latex)

# Salvar a tabela em um arquivo .tex
with open('resultados_analises.tex', 'w') as f:
    f.write(tabela_latex)

In [ ]:
# Criar uma instância da classe SemanticMatching
from semantic_matcher import ENPreprocessor, SemanticMatcher
tradutor = ENPreprocessor()
matcher = SemanticMatcher(curriculos_data, matriz_ceis_data, relacoes_biologicos, relacoes_pequenas_moleculas)

# Executar as etapas de processamento
matcher.traduzir_nomes_produtos()
matcher.extrair_caracteristicas()
matcher.classificar_produtos()
matcher.conectar_produtos_grafo()

# Imprimir informações sobre os grafos
print("Grafo de Biológicos:")
print("Número de nós:", matcher.biologicos.number_of_nodes())
print("Número de arestas:", matcher.biologicos.number_of_edges())

print("\nGrafo de Pequenas Moléculas:")
print("Número de nós:", matcher.pequenas_moleculas.number_of_nodes())
print("Número de arestas:", matcher.pequenas_moleculas.number_of_edges())

In [ ]:
import os
import cudf
import cugraph
import json
from pyvis.network import Network

json_folder = os.path.join(os.getcwd(),'_data','out_json')

# Carregar os dados dos arquivos JSON com cuDF
curriculos_df = cudf.read_json(os.path.join(json_folder,'curriculos.json'))

In [ ]:
curriculos_df.head()

In [ ]:
matriz_ceis_df.keys()

In [ ]:
import cudf
import cugraph
import json
import time
from pyvis.network import Network
from sklearn.metrics.pairwise import cosine_similarity

# Funções para calcular as transformadas de Fourier
def naive_fourier(x, gridsize=300):
    # ... (implementação da Naive Fourier Transform) ...
    pass  # Substitua pelo código da transformada

def gft(x, edge_index):
    # ... (implementação da Graph Fourier Transform) ...
    pass  # Substitua pelo código da transformada

def wavelet_transform(x, edge_index, num_scales=5):
    # ... (implementação da Wavelet Transform) ...
    pass  # Substitua pelo código da transformada

def fractional_fourier(x, alpha=0.5):
    # ... (implementação da Fractional Fourier Transform) ...
    pass  # Substitua pelo código da transformada

def qft(x):
  """Calcula a QFT de um quaternion."""
  # ... (implementação da Transformada de Fourier Quaterniônica) ...
  pass  # Substitua pelo código da transformada

def conectar_pesquisadores_produtos(graph, curriculos_df, matriz_ceis_df, tipo_transformada):
    """Conecta pesquisadores a produtos com base na similaridade de áreas de atuação."""

    # Extrair áreas de atuação dos pesquisadores e produtos
    pesquisadores_areas = curriculos_df['Áreas'].to_arrow().to_pylist()
    produtos_areas = []
    for bloco in matriz_ceis_df['blocos'].to_arrow().to_pylist():
        for produto in bloco['produtos']:
            produtos_areas.append(produto['nome'])  # Usando o nome do produto como representação da área

    # Converter áreas de atuação em vetores numéricos (ex: usando word embeddings)
    # ... (implementar lógica para converter áreas em vetores) ...

    # Aplicar a transformada de Fourier selecionada
    if tipo_transformada == 'naive':
        pesquisadores_transformados = [naive_fourier(areas) for areas in pesquisadores_areas]
        produtos_transformados = [naive_fourier(areas) for areas in produtos_areas]
    elif tipo_transformada == 'gft':
        # ... (aplicar GFT) ...
        pass
    elif tipo_transformada == 'wavelet':
        # ... (aplicar Wavelet Transform) ...
        pass
    elif tipo_transformada == 'fractional':
        # ... (aplicar Fractional Fourier Transform) ...
        pass
    elif tipo_transformada == 'qft':
        # ... (aplicar QFT) ...
        pass
    else:
        raise ValueError("Tipo de transformada inválido.")

    # Calcular a similaridade de cosseno entre os vetores transformados
    similaridade = cosine_similarity(pesquisadores_transformados, produtos_transformados)

    # Criar as arestas no grafo cuGraph com base na similaridade
    limite_similaridade = 0.8  # Definir um limite para a similaridade
    for i, pesquisador_id in enumerate(pesquisadores_ids):
        for j, produto_id in enumerate(produtos_ids):
            if similaridade[i, j] > limite_similaridade:
                G.add_edge(pesquisador_id, produto_id, weight=similaridade[i, j])

# --- Avaliação das abordagens ---

resultados = {}
tipos_transformadas = ['naive', 'gft', 'wavelet', 'fractional', 'qft']

for tipo_transformada in tipos_transformadas:
    inicio = time.time()

    # Criar um novo grafo para cada tipo de transformada
    G = cugraph.Graph()
    G.add_nodes_from(pesquisadores_ids, tipo='pesquisador')
    G.add_nodes_from(produtos_ids, tipo='produto')

    # Conectar pesquisadores e produtos
    conectar_pesquisadores_produtos(G, curriculos_df, matriz_ceis_df, tipo_transformada)

    fim = time.time()
    tempo_execucao = fim - inicio

    # Calcular as métricas de avaliação
    num_arestas = G.number_of_edges()

    # ... (calcular precisão e recall usando um conjunto de dados de referência) ...

    resultados[tipo_transformada] = {
        'num_arestas': num_arestas,
        'tempo_execucao': tempo_execucao,
        # 'precisao': precisao,
        # 'recall': recall
    }

# Imprimir os resultados
print(resultados)

# --- Visualização (opcional) ---
# ... (criar gráficos com Altair para comparar as métricas) ...

In [ ]:

# Pré-processamento dos dados com cuDF
# (Extrair IDs e informações relevantes para os nós e arestas)
pesquisadores_ids = curriculos_df['Identificação']['ID Lattes'].to_arrow().to_pylist()
produtos_ids = []
for bloco in matriz_ceis_df['blocos'].to_arrow().to_pylist():
    for produto in bloco['produtos']:
        produtos_ids.append(produto['id'])

# Criar um grafo cuGraph
G = cugraph.Graph()

# Adicionar os nós ao grafo cuGraph
G.add_nodes_from(pesquisadores_ids, tipo='pesquisador')
G.add_nodes_from(produtos_ids, tipo='produto')

# Criar as arestas entre pesquisadores e produtos (definir critérios)
# ... (implementar lógica para conectar pesquisadores a produtos usando cuDF) ...
# EXEMPLO: conectar pesquisadores a produtos com base na similaridade de áreas de atuação

# Calcular métricas de grafo com cuGraph
degree_centrality = cugraph.degree_centrality(G)

# Converter o grafo cuGraph para NetworkX
graph_nx = G.to_networkx()

# Criar o grafo PyVis 'net' a partir do grafo NetworkX 'graph_nx'
net = Network(notebook=True, directed=True)
net.from_nx(graph_nx)

# Personalizar a visualização (opcional)
# ... (utilizar as métricas calculadas com cuGraph para definir o tamanho dos nós, cores, etc.) ...

# Mostrar o grafo na célula do Jupyter
net.show("graph.html")

In [ ]:
import os
import json
import networkx as nx

json_folder = os.path.join(os.getcwd(),'_data','out_json')

# Carregar os dados dos arquivos JSON
with open(os.path.join(json_folder,'curriculos.json'), 'r') as f:
    curriculos_data = json.load(f)
with open(os.path.join(json_folder,'matriz_ceis.json'), 'r') as f:
    matriz_ceis_data = json.load(f)

# Criar um grafo direcionado
graph = nx.DiGraph()

# Adicionar os pesquisadores como nós
for pesquisador in curriculos_data:
    graph.add_node(pesquisador['Identificação']['ID Lattes'], tipo='pesquisador', **pesquisador)

# Adicionar os produtos como nós
for bloco in matriz_ceis_data['blocos']:
    for produto in bloco['produtos']:
        graph.add_node(produto['id'], tipo='produto', **produto)

# Função para criar as arestas entre pesquisadores e produtos
def create_edges(graph, pesquisador_node, produto_node):
    """Cria arestas entre pesquisadores e produtos com base em suas propriedades."""
    pesquisador_data = pesquisador_node[1]
    produto_data = produto_node[1]

    # Lógica para conectar pesquisadores a produtos (EXEMPLO)
    # Verificar se as áreas de atuação do pesquisador são relevantes para o produto
    for area in pesquisador_data['Áreas'].values():
        if any(keyword in area for keyword in ["Biotecnologia", "Saúde", "Química", "Biologia"]):
            graph.add_edge(pesquisador_node[0], produto_node[0])
            return  # Criar apenas uma aresta por produto

# Iterar pelos nós e criar as arestas
for pesquisador_node in graph.nodes(data=True):
    if pesquisador_node[1]['tipo'] == 'pesquisador':
        for produto_node in graph.nodes(data=True):
            if produto_node[1]['tipo'] == 'produto':
                create_edges(graph, pesquisador_node, produto_node)

# Imprimir os primeiros 5 nós e arestas
print("Nós:", list(graph.nodes(data=True))[:5])
print("Arestas:", list(graph.edges(data=True))[:5])

# Mostrar o número de nós e arestas
print("Número de nós:", graph.number_of_nodes())
print("Número de arestas:", graph.number_of_edges())

# Mostrar os tipos de nós presentes no grafo
tipos_nos = set(no[1]['tipo'] for no in graph.nodes(data=True))
print("Tipos de nós:", tipos_nos)

# Mostrar as propriedades dos nós
print("Propriedades dos nós:")
for no in graph.nodes(data=True):
    print(no)

# Mostrar a distribuição das arestas entre os nós
# (Exemplo: calcular o grau de cada nó)
graus = dict(graph.degree())
print("Distribuição de graus dos nós:", graus)

In [8]:
# %pip install pyvis

In [ ]:
import networkx as nx
from pyvis.network import Network

net = Network(notebook=True, directed=True)  # notebook=True para exibir no Jupyter, directed=True para grafo direcionado

In [10]:
net.from_nx(graph)

In [ ]:
# Definir opções de layout
net.repulsion(node_distance=200, central_gravity=0.2)

# Configurar a física da visualização
net.show_buttons(filter_=['physics'])  # Mostrar botões para controlar a física

# Definir cores para os nós com base no tipo
for node in net.nodes:
    if node['tipo'] == 'pesquisador':
        node['color'] = 'blue'
    elif node['tipo'] == 'produto':
        node['color'] = 'green'

# Ajustar o tamanho dos nós com base no grau
degrees = dict(graph.degree())
for node in net.nodes:
    node['size'] = degrees[node['id']] * 5  # Aumentar o tamanho proporcionalmente ao grau

# net.show("graph.html")

In [ ]:
from IPython.display import HTML
# Exibir o conteúdo do arquivo HTML na célula do Jupyter
HTML(filename='graph.html')

In [ ]:
# import cudf
# import cugraph
# import json
# import networkx as nx
# from pyvis.network import Network
# from googletrans import Translator
# from sklearn.metrics.pairwise import cosine_similarity

# class SemanticMatching:
#     def __init__(self, curriculos_df, matriz_ceis_data, relacoes_biologicos,
#                  relacoes_pequenas_moleculas):
#         self.curriculos_df = curriculos_df
#         self.matriz_ceis_data = matriz_ceis_data
#         self.relacoes_biologicos = relacoes_biologicos
#         self.relacoes_pequenas_moleculas = relacoes_pequenas_moleculas
#         self.translator = Translator()
#         self.produtos_df = self.criar_dataframe_produtos()
#         self.biologicos, self.pequenas_moleculas = self.criar_grafos_relacionamentos()

#     def criar_dataframe_produtos(self):
#         json_folder = os.path.join(os.getcwd(),'_data','out_json')

#         # Carregar a Matriz CEIS
#         with open(os.path.join(json_folder,'matriz_ceis.json'), 'r') as f:
#             matriz_ceis_data = json.load(f)

#         # Extrair os dados dos produtos
#         produtos = []
#         for bloco in matriz_ceis_data['blocos']:
#             for produto in bloco['produtos']:
#                 produto['bloco_id'] = bloco['id']
#                 produto['bloco_nome'] = bloco['titulo']
#                 produtos.append(produto)

#         # Retornar o DataFrame cuDF
#         return cudf.DataFrame(produtos)

#     def criar_grafos_relacionamentos(self):
#         # ... (código para criar os grafos de biológicos e pequenas moléculas) ...
#         biologicos = nx.DiGraph()
#         for node in self.relacoes_biologicos["nodes"]:
#             biologicos.add_node(node['id'], **node)
#         for edge in self.relacoes_biologicos["edges"]:
#             biologicos.add_edge(edge['from'], edge['to'])

#         pequenas_moleculas = nx.DiGraph()
#         for node in self.relacoes_pequenas_moleculas["nodes"]:
#             pequenas_moleculas.add_node(node['id'], **node)
#         for edge in self.relacoes_pequenas_moleculas["edges"]:
#             pequenas_moleculas.add_edge(edge['from'], edge['to'])
#         return biologicos, pequenas_moleculas

#     def traduzir_nomes_produtos(self):
#         # ... (código para traduzir os nomes dos produtos) ...
#         self.produtos_df['nome_en'] = self.produtos_df['nome'].apply(
#             lambda x: self.translator.translate(x, dest='en').text)

#     def extrair_caracteristicas(self):
#         # ... (código para extrair características semânticas) ...
#         pass  # Implemente a extração de características aqui

#     def classificar_produtos(self):
#         # ... (código para classificar os produtos) ...
#         pass  # Implemente a classificação dos produtos aqui

#     def calcular_similaridade(self, produto, grafo, tipo_transformada):
#         # ... (código para calcular similaridade usando a abordagem especificada) ...
#         pass  # Implemente o cálculo de similaridade aqui

#     def conectar_produtos_grafo(self):
#         # ... (código para conectar os produtos aos grafos) ...
#         pass  # Implemente a conexão dos produtos aos grafos aqui

#     def avaliar_desempenho(self):
#         # ... (código para avaliar o desempenho das abordagens) ...
#         pass  # Implemente a avaliação de desempenho aqui

In [ ]:
import calendar

yy = 2024
print (f"O calendário do ano {yy} é:")
print (calendar.calendar(yy))

In [ ]:
# import calendar

# yy = 2024
# # mm = 10
# # print(calendar.month(yy,mm))

# for i in range(1,13):
#     print(calendar.month(yy,i))

In [ ]:
# import pandas as pd

# # Define the CSS style for the DataFrame with the text wrapping in the lines and occupying the entire width of the cell
# style = """
# <style>
# .dataframe {
#     width: 100%;
# }

# .dataframe td {
#     word-wrap: break-word;
#     white-space: pre-wrap;
# }
# </style>
# """

# # Divide a coluna de desafios em múltiplas colunas, separando os desafios por ponto e vírgula
# df_produtos_proximos.iloc[:, 0] = df_produtos_proximos.iloc[:, 0].str.split(';')

# # "Explode" o DataFrame para que cada desafio fique em uma linha separada
# df_exploded = df_produtos_proximos.explode(df_produtos_proximos.columns[0])

# # Calcula a frequência de cada desafio
# df_sum = df_exploded[df_produtos_proximos.columns[0]].value_counts()

# # Converte a Series para DataFrame e redefine o índice
# df = pd.DataFrame(df_sum).reset_index()

# # Renomeia as colunas para 'Desafio' e 'Total'
# df.columns = ['Desafio', 'Total']

# # Exibe o DataFrame resultante
# df